# Notebook 2: The Full 3D LNS System and its Implementation

**Series:** LNS: From Theory to High-Fidelity Turbulence Simulation

## Introduction
In Notebook 1, we developed a robust 1D Local Navier-Stokes (LNS) solver by incorporating higher-order spatial reconstruction (MUSCL-Hancock) and a semi-implicit treatment for stiff relaxation source terms. This provided the necessary numerical foundation for accuracy and stability. Now, we take the most significant step in this series: extending our framework to a full three-dimensional (3D) system. 

This transition is not merely an addition of two more spatial dimensions; it introduces substantial new complexity, primarily in the tensor algebra required for the stress evolution and the structure of the 3D solver itself. This notebook will focus on:
1.  **Deriving the complete 3D LNS equations:** We will explicitly write down the flux vectors and, critically, the full source terms for the deviatoric stress tensor using a proper objective time derivative (the Upper Convected Maxwell - UCM model will be our primary example).
2.  **Outlining the 3D Finite Volume framework:** We will discuss the structure of a 3D solver on a Cartesian grid, including multi-dimensional flux calculations.
3.  **Code Structure for a 3D Solver:** While a full, high-performance 3D implementation is beyond the scope of a single notebook document, we will provide a detailed Python code structure and key functions to illustrate how the 3D logic is implemented and how it builds upon our 1D work.
4.  **Conceptual Validation:** We will discuss validation strategies for a 3D LNS code using simple test cases like the 3D advection of a vortex.

This notebook serves as the architectural blueprint for the high-fidelity turbulence simulations planned for the remainder of the series.

## 1. The Full 3D LNS System Equations

The 3D LNS system requires a state vector $\mathbf{Q}$ that captures the full 3D velocity field, heat flux vector, and the symmetric deviatoric stress tensor. 

**State Vector $\mathbf{Q}$ in 3D (13 variables):**
To ensure a symmetric stress tensor $\mathbf{\sigma}'$ and enforce the traceless condition ($	ext{tr}(\mathbf{\sigma}')=0$), we track 5 independent components. A common choice is:
$$\mathbf{Q} = [\rho, m_x, m_y, m_z, E_T, q_x, q_y, q_z, \sigma'_{xx}, \sigma'_{yy}, \sigma'_{xy}, \sigma'_{xz}, \sigma'_{yz}]^T$$
where $m_i = \rho u_i$ and $\sigma'_{zz}$ can be derived from the traceless condition: $\sigma'_{zz} = -(\sigma'_{xx} + \sigma'_{yy})$.

The system remains:
$$\frac{\partial \mathbf{Q}}{\partial t} + \frac{\partial \mathbf{F}_x}{\partial x} + \frac{\partial \mathbf{F}_y}{\partial y} + \frac{\partial \mathbf{F}_z}{\partial z} = \mathbf{S}(\mathbf{Q}, \nabla \mathbf{Q})$$

### 1.1 3D Flux Vectors
The flux vectors are direct extensions of the 2D case. For example, the x-flux vector $\mathbf{F}_x$ is:
$$\mathbf{F}_x = \begin{bmatrix}
\rho u_x & (m_x) \\
\rho u_x^2 + p - \sigma'_{xx} & (m_x) \\
\rho u_x u_y - \sigma'_{xy} & (m_y) \\
\rho u_x u_z - \sigma'_{xz} & (m_z) \\
(E_T + p)u_x - (\mathbf{\sigma}' \cdot \mathbf{u})_x + q_x & (E_T) \\
u_x q_x & (q_x) \\
u_x q_y & (q_y) \\
u_x q_z & (q_z) \\
u_x \sigma'_{xx} & (\sigma'_{xx}) \\
u_x \sigma'_{yy} & (\sigma'_{yy}) \\
u_x \sigma'_{xy} & (\sigma'_{xy}) \\
u_x \sigma'_{xz} & (\sigma'_{xz}) \\
u_x \sigma'_{yz} & (\sigma'_{yz})
\end{bmatrix}$$
where $(\mathbf{\sigma}' \cdot \mathbf{u})_x = \sigma'_{xx}u_x + \sigma'_{xy}u_y + \sigma'_{xz}u_z$. The flux vectors $\mathbf{F}_y$ and $\mathbf{F}_z$ follow by cyclic permutation.

### 1.2 The 3D Source Term $\mathbf{S}(\mathbf{Q})$: The Crux of the Complexity
The main challenge in 3D is the source term for the stress tensor components, which arises from the objective time derivative. We will use the **Upper Convected Maxwell (UCM)** model as our example. Its evolution equation is:
$$\tau_\sigma \frac{\mathcal{D}_\sigma \mathbf{\sigma}'}{\mathcal{D} t} + \mathbf{\sigma}' = \mathbf{\sigma}'_{NSF}$$
where the UCM objective derivative is:
$$\frac{\mathcal{D}_\sigma \mathbf{\sigma}'}{\mathcal{D} t} = \frac{\partial \mathbf{\sigma}'}{\partial t} + (\mathbf{u} \cdot \nabla)\mathbf{\sigma}' - \mathbf{L} \cdot \mathbf{\sigma}' - \mathbf{\sigma}' \cdot \mathbf{L}^T$$
and $\mathbf{L} = \nabla \mathbf{u}$ is the velocity gradient tensor.

Rearranging for our conservation law form $\frac{\partial \sigma'_{ij}}{\partial t} + \nabla \cdot (\mathbf{u} \sigma'_{ij}) = S_{\sigma'_{ij}}$, the source term for a component $\sigma'_{ij}$ is:
$$S_{\sigma'_{ij}} = \underbrace{\sigma'_{ij} (\nabla \cdot \mathbf{u})}_{\text{from advection}} + \frac{1}{\tau_\sigma} \left( \underbrace{\mathbf{L} \cdot \mathbf{\sigma}' + \mathbf{\sigma}' \cdot \mathbf{L}^T}_{\text{UCM stretching}} - \underbrace{\frac{1}{\tau_\sigma}(\mathbf{\sigma}' - \mathbf{\sigma}'_{NSF})}_{\text{relaxation}} \right)_{ij}$$

**Example: Source term for $\sigma'_{xx}$ (component $Q_8$):**
$$S_8 = \sigma'_{xx}(\nabla \cdot \mathbf{u}) + \frac{1}{\tau_\sigma}(\sigma'_{xx,NSF} - \sigma'_{xx}) + \frac{1}{\tau_\sigma} \left[ 2 \left( \frac{\partial u_x}{\partial x}\sigma'_{xx} + \frac{\partial u_x}{\partial y}\sigma'_{xy} + \frac{\partial u_x}{\partial z}\sigma'_{xz} \right) \right]$$

**Example: Source term for $\sigma'_{xy}$ (component $Q_{10}$):**
$$S_{10} = \sigma'_{xy}(\nabla \cdot \mathbf{u}) + \frac{1}{\tau_\sigma}(\sigma'_{xy,NSF} - \sigma'_{xy}) + \frac{1}{\tau_\sigma} \left[ \frac{\partial u_x}{\partial x}\sigma'_{xy} + \frac{\partial u_x}{\partial y}\sigma'_{yy} + \frac{\partial u_x}{\partial z}\sigma'_{yz} + \frac{\partial u_y}{\partial x}\sigma'_{xx} + \frac{\partial u_y}{\partial y}\sigma'_{xy} + \frac{\partial u_y}{\partial z}\sigma'_{yz} \right]$$

The source terms for heat flux $\mathbf{q}$ are simpler but follow the same principle. This explicit formulation is complex but essential for capturing the correct material response in 3D flows with rotation and strain.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# --- 3D System Definitions ---
NUM_VARS_3D = 13
# Q = [rho, mx, my, mz, E_T, qx, qy, qz, s_xx, s_yy, s_xy, s_xz, s_yz]

# --- Global Fluid & LNS Parameters (for demonstration) ---
GAMMA = 1.4; R_GAS = 287.0; CV_GAS = R_GAS / (GAMMA - 1.0)
MU_VISC = 1.8e-5; K_THERM = 0.026
TAU_Q = 1e-6; TAU_SIGMA = 1e-6

def Q_to_P_3D_enh(Q_vec):
    # ... (Implementation would extend 1D version to 3D velocities)
    # Returns P = [rho, ux, uy, uz, p, T]
    pass

def flux_3D_LNS_enh(Q_vec, direction='x'):
    # ... (Implementation of the 3D flux vector F_x, F_y, or F_z)
    pass

def source_3D_LNS_enh(Q_cell, grad_u_tensor, grad_T_vector):
    """Conceptual: Computes the full 3D source term vector.
    This is the most complex function to implement.
    Args:
        Q_cell: State vector at the cell center.
        grad_u_tensor: The 3x3 velocity gradient tensor L = grad(u) at the cell center.
        grad_T_vector: The 3x1 temperature gradient vector at the cell center.
    """
    S = np.zeros(NUM_VARS_3D)
    P = Q_to_P_3D_enh(Q_cell)
    rho, ux, uy, uz, p, T = P
    q = Q_cell[5:8] # qx, qy, qz
    s_xx, s_yy, s_xy, s_xz, s_yz = Q_cell[8:13]
    s_zz = -(s_xx + s_yy)
    
    # Reconstruct full stress tensor
    sigma_prime = np.array([[s_xx, s_xy, s_xz], [s_xy, s_yy, s_yz], [s_xz, s_yz, s_zz]])
    
    # Velocity gradient tensor L
    L = grad_u_tensor
    div_u = np.trace(L)
    
    # --- Heat Flux Source Terms (simplified objective deriv for now) ---
    q_nsf = -K_THERM * grad_T_vector
    if TAU_Q > 1e-12:
        # S_q = -(1/tau_q)*(q - q_nsf) + q*(div_u) - L^T.q etc.
        S[5:8] = -(1/TAU_Q) * (q - q_nsf) + q * div_u # Simplified version

    # --- Stress Source Terms (UCM Model) ---
    # NSF stress tensor
    strain_rate_tensor = 0.5 * (L + L.T)
    sigma_prime_nsf = 2 * MU_VISC * (strain_rate_tensor - (1/3) * div_u * np.identity(3))
    
    # UCM stretching terms
    ucm_stretching = np.dot(L, sigma_prime) + np.dot(sigma_prime, L.T)
    
    if TAU_SIGMA > 1e-12:
        # The source term for the full tensor sigma_prime is:
        S_sigma_tensor = sigma_prime * div_u + \
                         (1.0 / TAU_SIGMA) * (sigma_prime_nsf - sigma_prime) + \
                         (1.0 / TAU_SIGMA) * ucm_stretching
        
        # Assign components to the S vector
        S[8] = S_sigma_tensor[0,0] # S_xx
        S[9] = S_sigma_tensor[1,1] # S_yy
        S[10] = S_sigma_tensor[0,1] # S_xy
        S[11] = S_sigma_tensor[0,2] # S_xz
        S[12] = S_sigma_tensor[1,2] # S_yz
        
    return S

print("Conceptual 3D LNS System Definitions Outlined.")

Conceptual 3D LNS System Definitions Outlined.


## 2. 3D Finite Volume Framework

The extension of the FVM to 3D on a structured Cartesian grid is straightforward in principle. The update for a cell $(i,j,k)$ becomes:
$$\mathbf{Q}_{i,j,k}^{n+1} = \mathbf{Q}_{i,j,k}^n - \frac{\Delta t}{\Delta x} (\hat{\mathbf{F}}_{x,i+1/2,j,k} - \hat{\mathbf{F}}_{x,i-1/2,j,k}) - \frac{\Delta t}{\Delta y} (\hat{\mathbf{F}}_{y,i,j+1/2,k} - \hat{\mathbf{F}}_{y,i,j-1/2,k}) - \frac{\Delta t}{\Delta z} (\hat{\mathbf{F}}_{z,i,j,k+1/2} - \hat{\mathbf{F}}_{z,i,j,k-1/2}) + \Delta t \mathbf{S}_{i,j,k}^n$$

**Key Implementation Challenges:**
1.  **Data Structures:** State variables are now stored in 3D arrays (e.g., `Q[k, j, i, var_idx]`).
2.  **Flux Calculation:** Fluxes must be computed across faces in all three directions (x-faces, y-faces, z-faces). This typically involves 1D Riemann problems normal to each face.
3.  **Gradient Calculation:** The source terms require the full velocity gradient tensor $\nabla \mathbf{u}$ and temperature gradient $\nabla T$. These must be calculated accurately at each cell center, typically using finite differences on neighboring cell-averaged data, requiring a stencil of 27 cells for a central difference scheme.
4.  **Parallelization:** The computational cost of 3D simulations is immense. For any meaningful simulation (e.g., $128^3$ or larger), the domain must be decomposed and distributed across multiple processors using a parallelization library like **MPI (Message Passing Interface)**. Each processor handles a subdomain and communicates boundary information (ghost cell data) with its neighbors.

In [2]:
class LNS_Solver_3D_Conceptual:
    """A class to outline the structure of a 3D LNS solver."""
    def __init__(self, grid_dims, domain_size, cfl, fluid_params, lns_params):
        self.Nx, self.Ny, self.Nz = grid_dims
        self.Lx, self.Ly, self.Lz = domain_size
        self.dx, self.dy, self.dz = Lx/Nx, Ly/Ny, Lz/Nz
        self.cfl = cfl
        # ... store fluid and LNS params ...
        
        # Allocate memory for state vector Q (including ghost cells)
        self.Q = np.zeros((self.Nz + 2, self.Ny + 2, self.Nx + 2, NUM_VARS_3D))
        
    def set_initial_conditions(self, ic_func):
        # Loop over physical cells and apply IC
        pass
        
    def apply_boundary_conditions(self):
        # E.g., for periodic BCs in all directions
        # This is where MPI communication would happen in a parallel code
        pass
        
    def compute_gradients(self, Q_ghosted):
        # Compute grad(u) and grad(T) for all physical cells
        # This is a key and complex routine
        pass
        
    def compute_rhs_3d(self):
        # Implements the full 3D FVM update logic:
        # 1. Reconstruct interface states (e.g., MUSCL)
        # 2. Compute fluxes across x, y, z faces (e.g., HLLC)
        # 3. Compute source terms using pre-computed gradients
        # 4. Assemble the RHS = -div(F) + S
        pass
    
    def time_step(self, dt):
        # Apply a time integration scheme (e.g., RK3)
        # rhs1 = self.compute_rhs_3d()
        # Q1 = self.Q + dt * rhs1
        # ... etc for multi-stage methods
        pass

    def solve(self, t_final):
        # The main time-stepping loop
        # - Apply BCs
        # - Compute dt from CFL
        # - Call time_step() method
        # - Save data, print info
        pass

print("Structural outline for a 3D LNS solver class defined.")

Structural outline for a 3D LNS solver class defined.


## 3. Conceptual Validation: The 3D Advected Vortex

Before tackling full turbulence, a 3D LNS code must be validated. A standard test is the advection of an isentropic vortex (e.g., a Taylor-Green vortex or a manufactured solution).

**Setup:**
*   A smooth vortex solution is prescribed as the initial condition on a 3D periodic domain.
*   The vortex is given a uniform background advection velocity (e.g., $u=U_0, v=U_0, w=U_0$).
*   Since the vortex is a smooth solution to the Euler equations, dissipative effects should be minimal if the effective Reynolds number is high.

**Validation Metrics:**
1.  **Preservation of Vortex Structure:** The vortex should advect across the domain without significant numerical distortion or dissipation. This tests the accuracy of the advection scheme.
2.  **Conservation:** Global mass, momentum, and energy should be conserved to machine precision (or to the order of the time integration scheme) over time.
3.  **Convergence:** The error between the numerical solution and the exact analytical solution should decrease at the expected rate (e.g., second-order for a MUSCL scheme) as the grid is refined.

## 4. Conclusion and Path to Turbulence Simulation

This notebook has laid out the complete theoretical and structural blueprint for a 3D Local Navier-Stokes solver. We have:
*   **Explicitly defined the 3D LNS system**, including the complex but crucial source terms arising from the Upper Convected Maxwell model for stress relaxation. This completes the physical theory.
*   **Outlined the 3D Finite Volume framework**, highlighting the challenges of multi-dimensional flux calculations, gradient estimations, and the necessity of parallelization.
*   **Provided a conceptual code structure**, showing how the components (IC, BC, fluxes, sources, time-stepping) fit together in a high-level design.

**What has been achieved in this notebook is the final theoretical and architectural step before a full-scale implementation.** The path from here involves intensive software engineering, numerical methods development, and high-performance computing.

**The Next Notebooks in the Series will now leverage this (conceptually complete) 3D LNS solver to finally tackle the problem of turbulence directly:**

*   **Notebook 3: Simulating the Transition to Turbulence with LNS:** We will simulate the breakdown of a Taylor-Green vortex, a canonical path to turbulence, and analyze how LNS dynamics (viscoelasticity, finite thermal response) influence this transition compared to classical N-S.

*   **Notebook 4: Fully Developed Turbulence with LNS:** We will simulate Decaying Homogeneous Isotropic Turbulence (DHIT) to investigate the statistical properties of LNS turbulence, focusing on energy spectra and intermittency, and directly test the hypothesis that LNS provides a physically regularized description of small-scale motions.